In [1]:
import os
import nltk
import math
import random
import itertools
import collections

from nltk.corpus import brown
from nltk.classify import accuracy
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import NuSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [2]:
PARENT = '/Users/shreydesai/GitHub/niche'
CATEGORIES = ['entertainment', 'sports', 'fun', 'games', 
              'weather', 'science', 'technology', 'politics']

In [3]:
def fileids(category):
    """Get file IDs of tweets for specified category"""
    path = os.path.join(PARENT, 'corpus', 'processed', category)
    return os.listdir(path)

def sents(file):
    """Get list of sentences from a document"""
    f = open(file, 'r', encoding='ISO-8859-1').read().strip()
    return [sent.strip() for sent in f.split('\n')]

def words(file):
    """Get list of words from a document"""
    f = open(file, 'r', encoding='ISO-8859-1').read().strip()
    sents = [sent.split(' ') for sent in f.split('\n')]
    words = [word for sent in sents for word in sent if len(word) > 0]
    return words

In [4]:
# train tagge
brown_tagged_sents = brown.tagged_sents()
tagger = nltk.UnigramTagger(brown_tagged_sents, backoff=nltk.DefaultTagger('NN'))

In [10]:
stuff = ['PS', 'NN', 'OO', 'NN']
tags = nltk.bigrams(stuff)
list(tags)

[('PS', 'NN'), ('NN', 'OO'), ('OO', 'NN')]

In [ ]:
documents, total_words = [], []

for category in CATEGORIES:
    for fileid in fileids(category):
        if fileid == '.DS_Store':
            continue
        path = os.path.join(PARENT, 'corpus', 'processed',
                            category, fileid)
        
        doc_tags = []
        for sent in sents(path):
            #tags = ['{}_{}'.format(x,y) for (x,y) in tagger.tag(sent.split(' '))]
            tags = [y for (x,y) in tagger.tag(sent.split(' '))]
            #tags = list(nltk.bigrams(tags))
            doc_tags.extend(tags)
            total_words.extend(tags)
        documents.append((doc_tags, category))

print('POS tag count:', len(total_words))
print(total_words[:10])

In [ ]:
documents[0]

In [ ]:
def features(document):
    """Builds list of features for each document"""
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

def bigram_features(words, score_fn=BigramAssocMeasures.chi_sq, n=2000):
    """Builds bigram features from document"""
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    return {bigram: True for bigram in bigrams}

In [ ]:
'''
print('Building vocab')
vocab = set(total_words)
word_features = list(vocab)
random.shuffle(word_features)
word_features = word_features[:2000]

print('Creating features')
feature_sets = [(features(d), c) for (d, c) in documents]
random.shuffle(feature_sets)

print('Creating train/test sets')
cutoff = math.ceil(len(feature_sets) * 0.7)
train_set, test_set = feature_sets[:cutoff], feature_sets[cutoff:]
'''
feature_sets = [(bigram_features(d), c) for (d, c) in documents]

In [ ]:
feature_sets[0]

In [43]:
def display(num):
    return '{0:.2f}'.format(num)

def run_classifier(name, classifier, train_set, test_set):
    clf = SklearnClassifier(classifier)
    clf.train(train_set)
    
    # general classifier accuracy
    acc = accuracy(clf, test_set)
    print('{}: {}%'.format(name, display(acc * 100)))
    
    # confusion matrix
    gold = clf.classify_many([fs for (fs, l) in test_set])
    test = [l for (fs, l) in test_set]
    cm = nltk.ConfusionMatrix(gold, test)
    print(cm.pretty_format(show_percents=True))

# Naive Bayes
run_classifier('Multinomial NB', MultinomialNB(), train_set, test_set)
print()

# SVM
run_classifier('NuSVC', NuSVC(), train_set, test_set)
print()

# Logistic Regression
run_classifier('LogisticRegression', LogisticRegression(), train_set, test_set)

Multinomial NB: 82.98%
              |      e                                                  |
              |      n                                                  |
              |      t                                                  |
              |      e                                         t        |
              |      r                                         e        |
              |      t                    p                    c        |
              |      a                    o      s             h      w |
              |      i                    l      c      s      n      e |
              |      n             g      i      i      p      o      a |
              |      m             a      t      e      o      l      t |
              |      e      f      m      i      n      r      o      h |
              |      n      u      e      c      c      t      g      e |
              |      t      n      s      s      e      s      y      r |
--------------+